In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

In [ ]:
zip_path = "/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly-20241222T145439Z-001.zip"
extract_path = "/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data"

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Đã giải nén tại: {extract_path}")

Đã giải nén tại: /content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data


In [ ]:
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

In [ ]:
!pip install opencv-python-headless


In [ ]:
import cv2

In [ ]:
import os
from torch.utils.data import Dataset

class Butterfly(Dataset):
    def __init__(self, path, is_train, transform = None):
        self.classes = ["ADONIS", "AFRICAN GIANT SWALLOWTAIL", "AMERICAN SNOOT", "AN 88", "APPOLLO"]
        if is_train:
          data_path = os.path.join(path, "train")
        else:
          data_path = os.path.join(path, "valid")
        self.image_paths =[]
        self.labels =[]
        for label, class_name in enumerate(self.classes):
          sub_data_path= os.path.join(data_path, class_name)
          if not os.path.exists(sub_data_path):
                print(f"Warning: Path {sub_data_path} does not exist!")
                continue
          for image_name in os.listdir(sub_data_path):
            image_path = os.path.join(sub_data_path, image_name)
            self.image_paths.append(image_path)
            self.labels.append(label)
        self.transform = transform




    def __len__(self):
       return len(self.labels)

    def __getitem__(self, index):
       image = cv2.imread(self.image_paths[index])
       image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       if self.transform:
          image = self.transform(image)

       label = self.labels[index]
       return image, label




In [ ]:
from torchvision.transforms import Resize, ToTensor, Compose

In [18]:
import torch
import torch.nn as nn


In [19]:
class VGG16(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()

        # Block 1
        self.feature1 = nn.Sequential(
             nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
             nn.BatchNorm2d(num_features=64),
             nn.ReLU(),

             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
             nn.BatchNorm2d(num_features=64),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2),
            )
        self.feature2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),

            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )

        # Block 3

        self.feature3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.feature4 = nn.Sequential(
             nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
             nn.BatchNorm2d(num_features=512),
             nn.ReLU(),

             nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
             nn.BatchNorm2d(num_features=512),
             nn.ReLU(),

             nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
             nn.BatchNorm2d(num_features=512),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=2, stride=2),
           )
        # Block 5
        self.feature5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Fully Connected Layers
        self.fc1_fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=512 * 7 * 7, out_features=4096),
            nn.ReLU()
        )

        self.fc2_fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU()
        )

        self.fc3_fc = nn.Linear(in_features=4096, out_features=num_classes)

    def forward(self, x):
        x=self.feature1(x)
        x=self.feature2(x)
        x=self.feature3(x)
        x=self.feature4(x)
        x=self.feature5(x)
        # Flatten and FC layers
        x = x.view(x.shape[0], -1)  # Flatten
        x = self.fc1_fc(x)
        x = self.fc2_fc(x)
        x = self.fc3_fc(x)

        return x


In [ ]:
import torch.optim as optim

In [ ]:
from tqdm.autonotebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter

<ipython-input-9-e65bd15f6cb5>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
def train():
  num_epoch = 10
  batch_size =16
  tensorboard_dir = "/content/drive/MyDrive/Học máy/Phân loại các loài bướm/resu"
  checkpoints_dir ="/content/drive/MyDrive/Học máy/Phân loại các loài bướm/checkpoint"
  transform = Compose([
      ToTensor(),
      Resize((224,224))
  ])
  train_dataset =Butterfly(path="/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data/buttlefly", is_train =True, transform = transform)
  train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = 16,
    shuffle =True,
    num_workers= 2,
    drop_last = True
)
  val_dataset =Butterfly(path="/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data/buttlefly", is_train =False, transform = transform)
  val_dataloader = DataLoader(
    dataset = val_dataset,
    batch_size = 16,
    shuffle =False,
    num_workers= 2,
    drop_last = False
)

  model = VGG16(num_classes = len(train_dataset.classes))
  criterion = nn.CrossEntropyLoss()
  optimize = optim.SGD(params=model.parameters(), lr = 1e-3, momentum=0.9)
  if not os.path.isdir(tensorboard_dir):
    os.makedirs(tensorboard_dir)
  writer = SummaryWriter(tensorboard_dir)
  if not os.path.isdir(checkpoints_dir):
    os.makedirs(checkpoints_dir)
  best_acc = -1

  for epoch in range(num_epoch):
    model.train()
    progress_bar =tqdm(train_dataloader, colour= "red")
    for iter, (images, labels) in enumerate (progress_bar):
      outputs = model(images)
      loss = criterion(outputs,labels)
      progress_bar.set_description("Epoch {}/{}. Loss {:0.4f}".format(epoch +1 , num_epoch, loss))
      writer.add_scalar("Loss/train", loss, epoch*len(train_dataloader)+ iter)
      optimize.zero_grad()
      loss.backward()
      optimize.step()


    model.eval()
    losses = []
    all_labels = []
    all_predictions = []
    progress_bar = tqdm(val_dataloader, colour= "yellow")
    for iter, (images, labels) in enumerate(progress_bar):
      all_labels.extend(labels)
      outputs = model(images)
      predictons = torch.argmax(outputs, dim = 1)
      all_predictions.extend(predictons)
      loss = criterion(outputs, labels)
      losses.append(loss.item())
    avg_loss = np.mean(losses)
    avg_acc = accuracy_score(all_labels, all_predictions)
    print("Epoch {}/{}. Loss {:0.4f}".format(epoch +1 , num_epoch, avg_loss))
    print("Accuracy: {:0.4f}". format(avg_acc))
    writer.add_scalar("Loss/Valid", avg_loss, epoch)
    writer.add_scalar("Accuracy/Valid", accuracy_score(all_labels, all_predictions), epoch)
    check_point ={
        "epoch" : epoch+1,
        "model" :model.state_dict(),
        "optimize" : optimize.state_dict(),
        "best_acc" : best_acc


    }
    torch.save(check_point, os.path.join(checkpoints_dir, "last.pt".format(epoch)))
    if avg_acc > best_acc:
      torch.save(check_point, os.path.join(checkpoints_dir, "best.pt".format(epoch)))
      best_acc = avg_acc








In [ ]:
if __name__ == "__main__":
    train()

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/10. Loss 1.5474
Accuracy: 0.3200


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 9/10. Loss 0.0897
Accuracy: 0.9600


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 10/10. Loss 0.0278
Accuracy: 0.9600


In [21]:
transform = Compose([
      ToTensor(),
      Resize((224,224))
  ])
train_dataset =Butterfly(path="/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data/buttlefly", is_train =True, transform = transform)
train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = 16,
    shuffle =True,
    num_workers= 2,
    drop_last = True
)
val_dataset =Butterfly(path="/content/drive/MyDrive/Học máy/Phân loại các loài bướm/data/buttlefly_data/buttlefly", is_train =False, transform = transform)
val_dataloader = DataLoader(
    dataset = val_dataset,
    batch_size = 16,
    shuffle =False,
    num_workers= 2,
    drop_last = False
)

In [23]:

# Tải chỉ trọng số của mô hình
checkpoint = torch.load("/content/drive/MyDrive/Học máy/Phân loại các loài bướm/checkpoint/best.pt")
model.load_state_dict(checkpoint['model'], strict=False)  # strict=False cho phép bỏ qua các tham số không khớp
model.eval()

<ipython-input-23-fde9f5a79c9b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/MyDrive/Học máy/Phân loại các loài bướm/che

VGG16(
  (feature1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature3): Sequential(
    (0): Conv2d

In [27]:
import torch
from torchvision import transforms
from PIL import Image

# Tiền xử lý ảnh: Tương tự như khi huấn luyện mô hình
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Thay đổi kích thước ảnh (với VGG16 cần 224x224)
    transforms.ToTensor(),  # Chuyển ảnh thành tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Chuẩn hóa ảnh
])

# Đọc ảnh
img_path = "/content/Eighty-eight_Butterfly_(Diaethria_anna).JPG"
img = Image.open(img_path)

# Tiền xử lý ảnh
img_tensor = preprocess(img)
img_tensor = img_tensor.unsqueeze(0)  # Thêm một batch dimension (vì mô hình yêu cầu batch đầu vào)

# Đảm bảo rằng mô hình sử dụng GPU nếu có, nếu không thì sử dụng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
img_tensor = img_tensor.to(device)

# Dự đoán lớp
model.eval()  # Đặt mô hình ở chế độ đánh giá (tắt dropout, batch normalization)
with torch.no_grad():  # Tắt gradient tracking để tiết kiệm bộ nhớ
    output = model(img_tensor)

# Tính toán xác suất cho từng lớp (Sử dụng softmax để chuyển đổi output thành xác suất)
probabilities = torch.nn.functional.softmax(output, dim=1)

# Lấy lớp có xác suất cao nhất
_, predicted_class = torch.max(probabilities, 1)

# In kết quả dự đoán
class_idx = predicted_class.item()
print(f"Dự đoán lớp: {train_dataset.classes[class_idx]}")



Dự đoán lớp: AN 88
